In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test DFA-2-RegExp

In [2]:
// 06-Test-DFA-2-RegExp.ts
import { instance } from "@viz-js/viz";
import { RecursiveSet } from "recursive-set";
import {
  State,
  Char,
  DFA,
  RegExp,
  TransRelDet,
  key,
  dfa2regexp,
} from "./05-DFA-2-RegExp";
import { dfa2dot } from "./FSM-2-Dot";

In [3]:
// Singletons {1}, {2}, {3}
const S0: RecursiveSet<State> = new RecursiveSet<State>(1);
const S1: RecursiveSet<State> = new RecursiveSet<State>(2);
const S2: RecursiveSet<State> = new RecursiveSet<State>(3);

// Q = { {1}, {2}, {3} }
const Q: RecursiveSet<RecursiveSet<State>> =
  new RecursiveSet<RecursiveSet<State>>(S0, S1, S2);

// Σ = {a, b}
const Sigma: RecursiveSet<Char> = new RecursiveSet<Char>("a", "b");

// δ
const delta: TransRelDet = new Map<string, RecursiveSet<State>>();
delta.set(key(S0, "a"), S1); // (1,a)->2
delta.set(key(S1, "b"), S2); // (2,b)->3
delta.set(key(S2, "a"), S1); // (3,a)->2

// Startzustand und akzeptierende Zustände
const q0 = S0;
const A = new RecursiveSet<RecursiveSet<State>>(S2);

// DFA Objekt
const dfa: DFA = { Q, Sigma, delta, q0, A };

In [4]:
const { dot } = dfa2dot(dfa);
const viz = await instance();
display.html(viz.renderString(dot, { format: "svg" }));

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.1 (20251006.0113)
 -->
<!-- Title: Deterministic FSM Pages: 1 -->
 
 
 Deterministic FSM 
 
<!-- 1 -->
 
 1 
 
 
<!-- S0 -->
 
 S0 
 
 S0 
 
<!-- 1->S0 -->
 
 1->S0 
 
 
 
<!-- S1 -->
 
 S1 
 
 S1 
 
<!-- S0->S1 -->
 
 S0->S1 
 
 
 a 
 
<!-- S2 -->
 
 S2 
 
 
 S2 
 
<!-- S1->S2 -->
 
 S1->S2 
 
 
 b 
 
<!-- S2->S1 -->
 
 S2->S1 
 
 
 a

In [6]:
function prettyPrintRegExp(r: RegExp): string {
    if (typeof r === 'number' || typeof r === 'string') {
        return typeof r === 'string' ? `'${r}'` : String(r);
    }

    if (Array.isArray(r)) {
        const elements = (r as any[]).map((el: any) => prettyPrintRegExp(el)).join(", ");
        return `(${elements})`;
    }
    
    return String(r);
}

In [7]:
const r: RegExp = dfa2regexp(dfa);
prettyPrintRegExp(r)

(((0, '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', ('ε', '+', 0))), '+', ((('a', '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '⋅', ((('ε', '+', 0), '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '*')), '⋅', ('b', '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', ('ε', '+', 0))))), '+', ((((('ε', '+', 0), '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 0)), '+', ((('a', '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '⋅', ((('ε', '+', 0), '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '*')), '⋅', (0, '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', 0)))), '⋅', (((('ε', '+', 0), '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 0)), '+', ((('a', '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '⋅', ((('ε', '+', 0), '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '*')), '⋅', (0, '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', 0)))), '*')), '⋅', ((0, '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', ('ε', '+', 0))), '+', ((('a', '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '⋅', ((('ε', '+', 0), 

## Helpfunctions:

In [11]:
// -------------------------------
// Helpers
// -------------------------------
function isZero(r: RegExp): boolean {
  return r === 0;
}
function isEps(r: RegExp): boolean {
  return r === "ε" || r === "𝜀";
}
function eq(a: RegExp, b: RegExp): boolean {
  return JSON.stringify(a) === JSON.stringify(b);
}

As this regular expression is nearly unreadable,  The notebook `Rewrite.ipynb` contains the definition of the function `simplify` that can be used to simplify this expression.

In [14]:
function simplify(r: RegExp): RegExp {
  if (typeof r === "string" || typeof r === "number") return r;
  if (!Array.isArray(r)) return r;

  // ---------- Kleene Star ----------
  if (r.length === 2 && r[1] === "*") {
    let inner = simplify(r[0]);

    // 0* → ε
    if (isZero(inner)) return "ε";   

    // ε* → ε
    if (isEps(inner)) return "ε";    

    // (r*)* → r*
    if (Array.isArray(inner) && inner.length === 2 && inner[1] === "*") {
      return inner;
    }

    // (ε + r)* → r*
    if (
      Array.isArray(inner) &&
      inner.length === 3 &&
      inner[1] === "+" &&
      (isEps(inner[0]) || isZero(inner[0]))
    ) {
      const r2 = simplify(inner[2]);
      return [r2, "*"];
    }

    return [inner, "*"];
  }

  const left = simplify(r[0]);
  const op = r[1];
  const right = simplify(r[2]);

  // ------------------ +
  if (op === "+") {
    if (isZero(left)) return right;
    if (isZero(right)) return left;
    if (eq(left, right)) return left;
    return [left, "+", right];
  }

  // ------------------ ⋅
  if (op === "⋅") {
    if (isZero(left) || isZero(right)) return 0;
    if (isEps(left)) return right;
    if (isEps(right)) return left;
    return [left, "⋅", right];
  }

  return r;
}

In [15]:
const s = simplify(r);
prettyPrintRegExp(s)

(('a', '⋅', (('b', '⋅', 'a'), '*')), '⋅', 'b')


The function `regexp_2_string` takes a regular expression that is represented as a nested tuple and transforms it into a string.

In [17]:
function regexpToString(r: RegExp): string {
  // leere Sprache
  if (r === 0) return "0";

  // epsilon
  if (r === "ε" || r === "𝜀") return "ε";

  // einzelnes Symbol
  if (typeof r === "string") return r;

  // Array-Fälle
  if (Array.isArray(r)) {
    // -----------------------------------------
    // Binäre Operatoren: [r1, '⋅', r2] / [r1, '+', r2]
    // -----------------------------------------
    if (r.length === 3) {
      const [r1, op, r2] = r;

      if (op === "⋅") {
        return regexpToString(r1) + "⋅" + regexpToString(r2);
      }

      if (op === "+") {
        return "(" + regexpToString(r1) + "+" + regexpToString(r2) + ")";
      }
    }

    // -----------------------------------------
    // Kleene-Stern: [expr, '*']
    // -----------------------------------------
    if (r.length === 2 && r[1] === "*") {
      const inner = r[0];

      // Symbol* ohne Klammern
      if (typeof inner === "string") {
        return inner + "*";
      }

      // Komplexere Ausdrücke bekommen Klammern
      return "(" + regexpToString(inner) + ")*";
    }
  }

  throw new Error(`Invalid RegExp: ${JSON.stringify(r)}`);
}
console.log(regexpToString(s));

a⋅(b⋅a)*⋅b
